In [1]:
import pandas as pd
from neo4j import GraphDatabase
import pymongo

In [2]:
movies = pd.read_csv("./imdb/movies.csv")[:5000]

In [3]:
movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,NaN,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,537,$ 2250,NaN,NaN,NaN,7.0,7.0
1,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.9,171,NaN,NaN,NaN,NaN,4.0,2.0
2,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,420,$ 45000,NaN,NaN,NaN,24.0,3.0
3,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2019,NaN,NaN,NaN,NaN,28.0,14.0
4,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...","From the Manger to the Cross; or, Jesus of Naz...",1912,1913,"Biography, Drama",60,USA,English,Sidney Olcott,...,"R. Henderson Bland, Percy Dyer, Gene Gauntier,...","An account of the life of Jesus Christ, based ...",5.7,438,NaN,NaN,NaN,NaN,12.0,5.0


In [4]:
uri = "bolt://localhost:11003"
user = "neo4j"
password = "movies"
driver = GraphDatabase.driver(uri, auth=(user, password), encrypted=False)

In [5]:
session = driver.session()

In [6]:
def test_trans(tx, title):
    tx.run(
        'CREATE(m:Movie { title:$title }) RETURN m', title=title
    )

In [7]:
session.write_transaction(lambda tx: tx.run("MATCH (n) DETACH DELETE n"))

In [ ]:
for k,v in movies.iterrows():
    id = v.imdb_title_id
    title = v.title
    year = v.year
    original_title = v.original_title
    date_published = v.date_published
    genre = v.genre
    try:
        actor_names = v.actors.split(",")
    except:
        actor_names = []
        
    result = session.run("CREATE(m:Movie { title:$title, year:$year, original_title:$original_title , date_published:$date_published, genre:$genre }) RETURN m",
                        title=title, year=year, original_title=original_title, date_published=date_published, genre=genre)
    result = result.single()[0]
    
    for actor_name in actor_names:
        actor_name = actor_name.strip()
        match = session.run("match (a) WHERE a.name=$name return a",name=actor_name)
        if match.peek() == None:
            match = session.write_transaction(lambda tx, name: tx.run("CREATE(a:Name { name:$name } )RETURN a", name=name), actor_name)
        
        match = match.single()[0]
        result1 = session.run("""
            MATCH (name: Name{ name:$name })
            MATCH (movie: Movie{ title: $title })
            CREATE (movie)-[r1:FEATURES]->(name)
        """, name=actor_name, title=title)
        
   

In [ ]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

In [ ]:
imdb = myclient["imdb"]

In [ ]:
movie = imdb["movie"]
name = imdb["name"]

In [ ]:
for k,v in movies.iterrows():
    id = v.imdb_title_id
    title = v.title
    year = v.year
    original_title = v.original_title
    date_published = v.date_published
    genre = v.genre
    try:
        actor_names = v.actors.split(",")
        actor_names = list(map(lambda an: an.strip(), actor_names))
    except:
        actor_names = []
    
    movie.insert_one({
        "id":id,
        "title":title,
        "year": year,
        "original_title": original_title,
        "date_published": date_published,
        "genre": genre,
        "actor_names": actor_names
    })
    
    for actor_name in actor_names:
        actor_name = actor_name.strip()